# AWS Bedrock Phi3 inference

In [17]:
import boto3
import json

# Create Bedrock runtime client
client = boto3.client("sagemaker-runtime", region_name="us-east-1")

# Model ID for Phi-3 on Bedrock
MODEL_ID = "huggingface-llm-phi-3-mini-4k-instruct"

def build_chat_prompt(messages):
    """
    Convert OpenAI-style messages into a single prompt string.
    Example:
    [{"role": "system", "content": "You are a helpful assistant."},
     {"role": "user", "content": "Hello!"}]
    -->
    "System: You are a helpful assistant.\nUser: Hello!\nAssistant:"
    """
    prompt = ""
    for msg in messages:
        role = msg["role"].capitalize()
        content = msg["content"]
        prompt += f"{role}: {content}\n"
    prompt += "Assistant:"  # signal Phi-3 to continue
    return prompt

def get_chatbot_response(client, model_id, messages, temperature=0.7, top_p=0.9, max_tokens=512):
    # Build prompt from role+content messages
    prompt = build_chat_prompt(messages)

    payload = {
        "inputs": prompt,
        "parameters": {
            "temperature": temperature,
            "top_p": top_p,
            "max_new_tokens": max_tokens
        }
    }

    # Call Bedrock model
    response = client.invoke_model(
        modelId=model_id,
        body=json.dumps(payload),
        contentType="application/json",
        accept="application/json"
    )

    # Parse the response body
    result = json.loads(response["body"].read())

    # Hugging Face models return a list with generated text
    if isinstance(result, list):
        return result[0].get("generated_text", "").strip()
    return result.get("generated_text", "").strip()


In [18]:
# Example messages
messages = [
    {"role": "system", "content": "You are an expert IT consultant."},
    {"role": "user", "content": "What are the most important domains in IT currently?"}
]

# Get response
response = get_chatbot_response(client, MODEL_ID, messages)
print("Assistant:", response)

AttributeError: 'SageMakerRuntime' object has no attribute 'invoke_model'